In [3]:
import tensorflow as tf
from cpsrl.models.mean import ConstantMean, LinearMean
from cpsrl.models.covariance import EQ
from cpsrl.models.gp import VFEGP
from cpsrl.models.gp import VFEGPStack

In [5]:
# Check constant mean works
constant_mean = ConstantMean(input_dim=3,
                             trainable=True,
                             dtype=tf.float64)

ones = tf.ones(shape=(2, 3), dtype=tf.float64)
constant_mean(ones)

# Check linear mean works
linear_mean = LinearMean(input_dim=3,
                         trainable=True,
                         dtype=tf.float64)
linear_mean(ones)

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[0.],
       [0.]])>

In [6]:
# Check EQ covariance works
log_coeff = 0.
log_scales = [-1., -1.]

# Check EQ kernel works
eq = EQ(log_coeff=log_coeff,
        log_scales=log_scales,
        trainable=True,
        dtype=tf.float64)

x1 = tf.random.uniform(shape=(4, 2), dtype=tf.float64)
x2 = tf.random.uniform(shape=(3, 2), dtype=tf.float64)

eq(x1, x2)

<tf.Tensor: shape=(4, 3), dtype=float64, numpy=
array([[0.58096107, 0.29320622, 0.90369351],
       [0.60384383, 0.16788348, 0.70138376],
       [0.6864302 , 0.0219259 , 0.25855786],
       [0.20834818, 0.42527721, 0.6887336 ]])>

In [7]:
# Check VFEGP works
dtype = tf.float64
trainable_mean = True
trainable_cov = True
trainable_noise = True
trainable_inducing = True
log_coeff = -1.
log_scales = [-1., -1., -1.]
log_noise = -4.

num_data = 10
state_dim = 2
action_dim = 1
num_ind = 10

x_train = tf.random.uniform(shape=(num_data, state_dim + action_dim), 
                            dtype=dtype)
y_train = tf.random.uniform(shape=(num_data, state_dim),
                            dtype=dtype)

mean = LinearMean(input_dim=(state_dim+action_dim),
                  trainable=trainable_mean,
                  dtype=dtype)

cov = EQ(log_coeff=log_coeff,
         log_scales=log_scales,
         trainable=trainable_cov,
         dtype=dtype)

vfe_gp = VFEGP(mean=mean,
               cov=cov,
               x_train=x_train,
               y_train=y_train[:, :1],
               x_ind=None,
               num_ind=num_ind,
               trainable_inducing=trainable_inducing,
               log_noise=log_noise,
               trainable_noise=trainable_noise,
               dtype=dtype) 

vfe_gp.free_energy()

<tf.Tensor: shape=(), dtype=float64, numpy=-1073.5908051557456>

In [8]:
# Check VFEGP works
dtype = tf.float64
trainable_mean = True
trainable_cov = True
trainable_noise = True
trainable_inducing = True
log_coeff = -1.
log_scales = [-1., -1., -1.]
log_noise = -4.

num_data = 10
state_dim = 2
action_dim = 1
num_ind = 10

x_train = tf.random.uniform(shape=(num_data, state_dim + action_dim), 
                            dtype=dtype)
y_train = tf.random.uniform(shape=(num_data, state_dim),
                            dtype=dtype)

means = [LinearMean(input_dim=(state_dim+action_dim),
                    trainable=trainable_mean,
                    dtype=dtype) for i in range(state_dim)]

covs = [EQ(log_coeff=log_coeff,
           log_scales=log_scales,
           trainable=trainable_cov,
           dtype=dtype) for i in range(state_dim)]

vfe_gps = [VFEGP(mean=means[i],
                 cov=covs[i],
                 x_train=x_train,
                 y_train=y_train[:, i:i+1],
                 x_ind=None,
                 num_ind=num_ind,
                 trainable_inducing=trainable_inducing,
                 log_noise=log_noise,
                 trainable_noise=trainable_noise,
                 dtype=dtype) 
           for i in range(state_dim)]

vfe_gp_stack = VFEGPStack(vfe_gps=vfe_gps,
                          dtype=dtype)

vfe_gp_stack.free_energy()

<tf.Tensor: shape=(), dtype=float64, numpy=-1010.5549664707199>

In [9]:
num_steps = 100
optimizer = tf.keras.optimizers.Adam(1e-1)

for step in range(num_steps + 1):
    
    with tf.GradientTape() as tape:
        
        tape.watch(vfe_gp_stack.trainable_variables)

        free_energy = vfe_gp_stack.free_energy()
        loss = - free_energy
        print(loss)
        
    gradients = tape.gradient(loss, vfe_gp_stack.trainable_variables)
    optimizer.apply_gradients(zip(gradients, vfe_gp_stack.trainable_variables))

tf.Tensor(1010.5549664707199, shape=(), dtype=float64)
tf.Tensor(558.6920394545932, shape=(), dtype=float64)
tf.Tensor(325.0667075820628, shape=(), dtype=float64)
tf.Tensor(191.37543676507306, shape=(), dtype=float64)
tf.Tensor(118.66852835194618, shape=(), dtype=float64)
tf.Tensor(82.26573799381734, shape=(), dtype=float64)
tf.Tensor(58.4297197049787, shape=(), dtype=float64)
tf.Tensor(44.491595847451904, shape=(), dtype=float64)
tf.Tensor(37.416523165001905, shape=(), dtype=float64)
tf.Tensor(34.00328723310163, shape=(), dtype=float64)
tf.Tensor(31.905027358912026, shape=(), dtype=float64)
tf.Tensor(30.12611530498993, shape=(), dtype=float64)
tf.Tensor(28.81365629475879, shape=(), dtype=float64)
tf.Tensor(28.1211221068358, shape=(), dtype=float64)
tf.Tensor(28.280235717501853, shape=(), dtype=float64)
tf.Tensor(28.53193900675546, shape=(), dtype=float64)
tf.Tensor(28.609004172712517, shape=(), dtype=float64)
tf.Tensor(28.777495460507293, shape=(), dtype=float64)
tf.Tensor(29.11168834

In [10]:
num_features = 100
x = tf.reshape(tf.range(15, dtype=dtype), (5, 3))

post_sample = vfe_gp_stack.sample_posterior(num_features=num_features)

In [11]:
post_sample(x, add_noise=True)

<dtype: 'float64'> float64
<dtype: 'float64'> float64


<tf.Tensor: shape=(5, 2), dtype=float64, numpy=
array([[-0.47611649, -0.1618677 ],
       [ 0.36558313, -0.06068338],
       [ 0.21869584, -0.32150761],
       [ 0.3022076 , -0.28994795],
       [ 0.1693977 ,  0.16263553]])>

In [12]:
vfe_gp_stack.add_training_data(x_train=x_train,
                               y_train=y_train)

In [15]:
tf.debugging.assert_near(vfe_gp.noise, 0.01831, atol=1e-2)


